In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('titanic').master('local').getOrCreate()

In [2]:
data = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [3]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [24]:
my_cols = data.select(
    ['Survived',
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Ticket',
    'Fare',
    'Embarked']
)

In [25]:
my_final_data = my_cols.na.drop()

In [26]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline

In [27]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [28]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [29]:
assembler = VectorAssembler(inputCols=[
    'Pclass',
    'SexVec',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'EmbarkVec'],outputCol='features')

In [30]:
from pyspark.ml.classification import LogisticRegression

In [31]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [32]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, 
                            gender_encoder, embark_encoder,
                            assembler,
                            log_reg_titanic])

In [33]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,0.3])

In [34]:
fit_model = pipeline.fit(train_titanic_data)

In [35]:
results = fit_model.transform(test_titanic_data)

In [36]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [37]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [38]:
AUC = my_eval.evaluate(results)
AUC

0.7425548589341693